<a href="https://colab.research.google.com/github/kdidi99/ml_heidelberg/blob/main/qda_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Team members:

*   Rudolf Braun
*   Jannik Schacht
*   Kieran Didi

# 1. Data preparation

# 1.1 Dimension Reduction

# 1.2 Scatterplot